Task0

In [148]:
from jedi.api.refactoring import inline
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Task1

In [149]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./mammographic_masses_data.csv"))
print(frame)

      BA   Age  Shape  Margin  Density  Severity
0    5.0  67.0    3.0     5.0      3.0         1
1    4.0  43.0    1.0     1.0      NaN         1
2    5.0  58.0    4.0     5.0      3.0         1
3    4.0  28.0    1.0     1.0      3.0         0
4    5.0  74.0    1.0     5.0      NaN         1
..   ...   ...    ...     ...      ...       ...
956  4.0  47.0    2.0     1.0      3.0         0
957  4.0  56.0    4.0     5.0      3.0         1
958  4.0  64.0    4.0     5.0      3.0         0
959  5.0  66.0    4.0     5.0      3.0         1
960  4.0  62.0    3.0     3.0      3.0         0

[961 rows x 6 columns]


In [150]:
import plotly.express as px

#2.1: Is de basic funtie van summary in dataframe (describe() voldoende?)
# Main functionality of pandas is reading csv files using (pd.read_csv()) 
# and converting them to a dataframe(pd.DataFrame()). to Further manipulate and visualize the data.using pandas or other libraries like plotlygh.
# Another more specific function of pandas is to fill in missing values. df.fillna(0) will replace all NaN values with 0.

# For a general look at what is inside the dataframe, pandas.DataFrame.info() works well, but for summary statistics one should use pandas.DataFrame.describe().
#print(frame.info())
print(frame.describe())

#2.2
# Makes a histogram of the Margin values of the rows in the database where Severity = 1
fig = px.histogram(frame.loc[frame.Severity == 1].Margin, x="Margin", title="Histogram of Margin with Severity = 1")
fig.show()

#2.3
# A pie-chart showing how many cases of severity = 1 were found per shape category
fig = px.pie(frame, values='Severity', names='Shape', title='Severity per Shape')
fig.show()

# A 2D heatmap with shape and severity on the axis, and the warmth depending on the frequency, accompanied by two individual histograms of the axes
fig = px.density_heatmap(frame, x="Shape", y="Severity", marginal_x="histogram", marginal_y="histogram", title="Density Heatmap of Shape and Severity")
fig.show()

# A scatter plot with on the axes the BA and Age values, and the color depending on the severity
fig = px.scatter(frame, x="BA", y="Age", color="Severity", title="Scatter Plot of BA and Age")
fig.show()

               BA         Age       Shape      Margin     Density    Severity
count  959.000000  956.000000  930.000000  913.000000  885.000000  961.000000
mean     4.300313   55.487448    2.721505    2.796276    2.910734    0.463059
std      0.683469   14.480131    1.242792    1.566546    0.380444    0.498893
min      0.000000   18.000000    1.000000    1.000000    1.000000    0.000000
25%      4.000000   45.000000    2.000000    1.000000    3.000000    0.000000
50%      4.000000   57.000000    3.000000    3.000000    3.000000    0.000000
75%      5.000000   66.000000    4.000000    4.000000    3.000000    1.000000
max      6.000000   96.000000    4.000000    5.000000    4.000000    1.000000


Task3 - NOT FINISHED

In [151]:
copy = frame.copy()

# 3.1
# clean the set by removing the rows containing missing data
copy = copy.dropna()

# making an histogram of both before and after removing incomplete rows
import plotly.graph_objects as go

fig = go.Figure()
fig = fig.add_trace(go.Histogram(x = frame.loc[frame.Severity == 1].Margin, name = "Original Set"))
fig = fig.add_trace(go.Histogram(x = copy.loc[copy.Severity == 1].Margin, name = "Cleaned Set"))
fig.show()

# 3.2
# 3.2
max =  -100000000000
min = 1000000000
normalized = frame.copy()

for i in frame.Age:
    if i > max:
        max = i
    if i < min:
        min = i

for i,j in enumerate(normalized.Age):
    normalized.loc[i, "Age"] = 1/(max-min) * j

print(normalized)

      BA       Age  Shape  Margin  Density  Severity
0    5.0  0.858974    3.0     5.0      3.0         1
1    4.0  0.551282    1.0     1.0      NaN         1
2    5.0  0.743590    4.0     5.0      3.0         1
3    4.0  0.358974    1.0     1.0      3.0         0
4    5.0  0.948718    1.0     5.0      NaN         1
..   ...       ...    ...     ...      ...       ...
956  4.0  0.602564    2.0     1.0      3.0         0
957  4.0  0.717949    4.0     5.0      3.0         1
958  4.0  0.820513    4.0     5.0      3.0         0
959  5.0  0.846154    4.0     5.0      3.0         1
960  4.0  0.794872    3.0     3.0      3.0         0

[961 rows x 6 columns]


Task4 - NOT FINISHED

In [181]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
# 4.1
#split copy to input=everything but severity and output=severity
copy_41 = copy.copy()
output = copy_41["Severity"]
input = copy_41.drop("Severity", axis=1)
#selectfrommodel part
selector = SelectFromModel(estimator=LogisticRegression()).fit_transform(input, output)
print(selector.get_feature_names_out(input))
#new plot
fig = go.Figure()
fig = fig.add_trace(go.Histogram(x = frame.loc[frame.Severity == 1].Margin, name = "Original Set"))
fig = fig.add_trace(go.Histogram(x = selector.loc[selector.Severity == 1].Margin, name = "Cleaned Set"))
fig.show()

# 4.2 - Plotten
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
components = pca.fit_transform(copy)
print(components)
fig = px.scatter(components, x=0, y=1)
fig.show()

# 4.3 Plotten
from sklearn.decomposition import TruncatedSVD
X = copy.copy()
svd = TruncatedSVD(n_components=2, n_iter=9, random_state=20)
reduced = svd.fit_transform(X)
print(reduced)

[[ 1.13226858e+01 -1.35485108e+00  1.06453403e+00]
 [ 2.37156222e+00 -2.46867038e+00  2.80779796e-01]
 [-2.78849587e+01  9.26204352e-01  3.47971128e-01]
 ...
 [ 8.32980335e+00 -1.81152183e+00  4.60601195e-01]
 [ 1.03570934e+01 -1.99375378e+00  2.83593344e-01]
 [ 6.20978419e+00  2.17405400e-01  1.96187604e-02]]


[[67.49079013 -1.05008076]
 [58.59640663 -2.49224057]
 [28.3598079  -0.33537437]
 ...
 [64.47590323 -1.48013312]
 [66.54560941 -1.72133893]
 [62.34049594  0.34660182]]


      BA   Age  Shape  Margin  Density  Severity
0    5.0  67.0    3.0     5.0      3.0         1
2    5.0  58.0    4.0     5.0      3.0         1
3    4.0  28.0    1.0     1.0      3.0         0
8    5.0  57.0    1.0     5.0      3.0         1
10   5.0  76.0    1.0     4.0      3.0         1
..   ...   ...    ...     ...      ...       ...
956  4.0  47.0    2.0     1.0      3.0         0
957  4.0  56.0    4.0     5.0      3.0         1
958  4.0  64.0    4.0     5.0      3.0         0
959  5.0  66.0    4.0     5.0      3.0         1
960  4.0  62.0    3.0     3.0      3.0         0

[830 rows x 6 columns]
TruncatedSVD(n_iter=7, random_state=20)
[[67.49079013 -1.05008076]
 [58.59640663 -2.49224057]
 [28.3598079  -0.33537437]
 ...
 [64.47590323 -1.48013312]
 [66.54560941 -1.72133893]
 [62.34049594  0.34660182]]
